In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv("../Data/train.csv");
test=pd.read_csv("../Data/test.csv");

In [3]:
# Impute missing values using the median for numeric columns and the most common value for string columns.
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [18]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
708,709,1,1,"Cleaver, Miss. Alice",female,22.0,0,0,113781,151.55,NaN,S
506,507,1,2,"Quick, Mrs. Frederick Charles (Jane Richards)",female,33.0,0,2,26360,26.00,NaN,S
559,560,1,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,36.0,1,0,345572,17.40,NaN,S


In [19]:
feature_cols=["Pclass","Sex","Age","Parch","Fare"] #"SibSp"
nonnumeric_cols=["Sex"]

In [20]:
# Join train and test data before imputing missing values in case test distribution is different
combined_X=train[feature_cols].append(test[feature_cols])
combined_imputed_X = DataFrameImputer().fit_transform(combined_X)

In [21]:
combined_imputed_X[col]

0        male
1      female
2      female
3      female
4        male
5        male
6        male
7        male
8      female
9      female
10     female
11     female
12       male
13       male
14     female
15     female
16       male
17       male
18     female
19     female
20       male
21       male
22     female
23       male
24     female
25     female
26       male
27       male
28     female
29       male
        ...  
388      male
389      male
390      male
391    female
392      male
393      male
394      male
395    female
396      male
397    female
398      male
399      male
400    female
401      male
402    female
403      male
404      male
405      male
406      male
407      male
408    female
409    female
410    female
411    female
412    female
413      male
414    female
415      male
416      male
417      male
Name: Sex, Length: 1309, dtype: object

In [29]:
le=LabelEncoder()
for col in nonnumeric_cols:
    combined_imputed_X[col] = le.fit_transform(combined_imputed_X[col])

In [30]:
combined_imputed_X[col]

0      1
1      0
2      0
3      0
4      1
5      1
6      1
7      1
8      0
9      0
10     0
11     0
12     1
13     1
14     0
15     0
16     1
17     1
18     0
19     0
20     1
21     1
22     0
23     1
24     0
25     0
26     1
27     1
28     0
29     1
      ..
388    1
389    1
390    1
391    0
392    1
393    1
394    1
395    0
396    1
397    0
398    1
399    1
400    0
401    1
402    0
403    1
404    1
405    1
406    1
407    1
408    0
409    0
410    0
411    0
412    0
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 1309, dtype: int64

In [31]:
# Prepare inputs for the model. Convert into numpy array
train_X=combined_imputed_X[0:train.shape[0]].as_matrix()
test_X=combined_imputed_X[train.shape[0]:].as_matrix()
train_y=train["Survived"]

In [34]:
# Build the Model using XGBoost
# Number of decision trees is controlled by n_estimators argument which is 100 by default.
model=xgb.XGBClassifier(max_depth=3,n_estimators=300,learning_rate=0.05).fit(train_X,train_y)
predict_y=model.predict(test_X)

In [35]:
output=pd.DataFrame({"PassengerId":test["PassengerId"],"Survived":predict_y})

In [36]:
output.to_csv("output.csv",index=False)